In [1]:
!pip install geopandas

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json

def setEdj(df: pd.DataFrame, key) -> None :
    df.loc[df[key].isin(['5', '6', '7']), key] = 'VET'
    df.loc[df[key].isin(['8', '9']), key] = 'GE'
    df.loc[df[key].isin(['2', '3', '4']), key] = 'IS'
    df.loc[df[key].isin(['1']), key] = 'NET'

In [3]:
merged_df = pd.read_csv("./study-data/TREE2_Data_IWI_Open_Data_Vorlesung_2023_nolabel.csv")

merged_df = merged_df[merged_df['t1bqvalids'] != 'No valid response']
merged_df = merged_df[merged_df['t2bqvalids'] != 'No valid response']
merged_df = merged_df[merged_df['t3bqvalids'] != 'No valid response']

merged_df.dropna(inplace=True)
merged_df['t1educ_class_1_r'] = merged_df['t1educ_class_1_r'].astype(int).astype(str)
merged_df['t2educ_class_1_r'] = merged_df['t2educ_class_1_r'].astype(int).astype(str)
merged_df['t3educ_class_1_r'] = merged_df['t3educ_class_1_r'].astype(int).astype(str)


setEdj(merged_df, 't1educ_class_1_r')
setEdj(merged_df, 't2educ_class_1_r')
setEdj(merged_df, 't3educ_class_1_r')

cols = ['resp_id', 'aes_canton', 'aes_langreg', 't1wt', 't2wt', 't3wt', 't1educ_class_1_r', 't2educ_class_1_r', 't3educ_class_1_r']

merged_df = merged_df[cols]
merged_df.fillna(0, inplace=True)

merged_df.head(20)


,resp_id,aes_canton,aes_langreg,t1wt,t2wt,t3wt,t1educ_class_1_r,t2educ_class_1_r,t3educ_class_1_r
1,1011011,10,2,4.959451,5.249187,4.995442,IS,VET,VET
3,1011013,11,1,9.184840,9.340135,13.201428,VET,VET,NET
6,1011020,15,1,0.612540,0.618889,0.562086,VET,VET,VET
7,1011021,10,1,10.286020,14.506459,10.028031,VET,VET,VET
8,1011027,10,1,2.655471,2.107327,2.090831,GE,GE,GE
10,1011029,25,2,3.366786,3.206607,3.349368,GE,GE,GE
11,1011030,15,1,0.662122,0.778710,0.708539,IS,VET,VET
12,1011033,25,2,11.310630,9.486566,35.055328,IS,NET,NET
13,1011035,1,1,17.446143,15.878919,16.129011,VET,VET,VET
15,1011037,17,1,3.042590,2.854162,2.945887,GE,GE,GE


In [6]:
# Grouping data by canton and education type and calculating the weighted count
w1_cantons = merged_df.groupby(['aes_canton', 't1educ_class_1_r']).apply(
    lambda x: (x['t1wt']).sum()).reset_index(name='w1_weighted_count')

w2_cantons = merged_df.groupby(['aes_canton', 't2educ_class_1_r']).apply(
    lambda x: (x['t1wt']).sum()).reset_index(name='w2_weighted_count')

w3_cantons = merged_df.groupby(['aes_canton', 't3educ_class_1_r']).apply(
    lambda x: (x['t1wt']).sum()).reset_index(name='w3_weighted_count')


w1_cantons = w1_cantons.rename(columns={'t1educ_class_1_r': 'education'})
w2_cantons = w2_cantons.rename(columns={'t2educ_class_1_r': 'education'})
w3_cantons = w3_cantons.rename(columns={'t3educ_class_1_r': 'education'})

merged_df = pd.merge(w1_cantons, w2_cantons, on=['aes_canton', 'education'], how='outer')
merged_df = pd.merge(merged_df, w3_cantons, on=['aes_canton', 'education'], how='outer')

#merged_df.set_index('education', inplace=True)
merged_df.head(20)

KeyError: 't1educ_class_1_r'

In [ ]:
# TODO: df.to_html('./test.html')

In [ ]:
'''
Summarise the columns w1_weighted_count, w2_weighted_count, and w3_weighted_count across all aes_canton for each education category, 
and then append these summaries back to your original DataFrame.
'''

summary_df = merged_df.groupby('education').sum().reset_index()
summary_df.drop('aes_canton', axis=1, inplace=True)
summary_df['aes_canton'] = 0
final_df = pd.concat([merged_df, summary_df], ignore_index=True)

final_df.to_csv('./study-data/currated_data.csv')
final_df.head(20)

,aes_canton,education,w1_weighted_count,w2_weighted_count,w3_weighted_count
0,1,GE,2822.459832,2875.675016,2792.182829
1,1,IS,1717.171951,330.527861,114.166294
2,1,NET,340.946749,165.893318,493.385885
3,1,VET,4949.327069,6457.809406,6430.170593
4,2,GE,2234.584649,2285.406745,2264.229984
5,2,IS,1346.252183,183.111332,81.462125
6,2,NET,123.627758,106.999650,208.084862
7,2,VET,3442.092653,4571.039517,4592.780272
8,3,GE,734.225355,786.724219,761.738568
9,3,IS,313.620559,87.777121,24.908537


In [ ]:
multi_index_df = final_df.set_index(['aes_canton', 'education'])
multi_index_df.to_csv('./study-data/currated_data.csv')
multi_index_df.loc[(4, 'IS')]

w1_weighted_count    8.090224
w2_weighted_count         NaN
w3_weighted_count    5.293744
Name: (4, IS), dtype: float64

In [ ]:
details = {}
for canton_id in multi_index_df.index.get_level_values('aes_canton').unique():
    canton_details = {}
    for edu_level in multi_index_df.xs(canton_id, level='aes_canton').index:
        canton_details[edu_level] = {
            'w1_weighted_count': multi_index_df.loc[(canton_id, edu_level), 'w1_weighted_count'],
            'w2_weighted_count': multi_index_df.loc[(canton_id, edu_level), 'w2_weighted_count'],
            'w3_weighted_count': multi_index_df.loc[(canton_id, edu_level), 'w3_weighted_count']
        }
    details[str(canton_id)] = canton_details
details

{'1': {'GE': {'w1_weighted_count': 2822.4598321753724,
   'w2_weighted_count': 2875.675016190895,
   'w3_weighted_count': 2792.1828294931406},
  'IS': {'w1_weighted_count': 1717.171950571976,
   'w2_weighted_count': 330.5278613141312,
   'w3_weighted_count': 114.16629368517553},
  'NET': {'w1_weighted_count': 340.94674926468315,
   'w2_weighted_count': 165.89331780402065,
   'w3_weighted_count': 493.38588503765476},
  'VET': {'w1_weighted_count': 4949.327068815885,
   'w2_weighted_count': 6457.809405518869,
   'w3_weighted_count': 6430.170592611945}},
 '2': {'GE': {'w1_weighted_count': 2234.584649055655,
   'w2_weighted_count': 2285.4067452956183,
   'w3_weighted_count': 2264.229984007583},
  'IS': {'w1_weighted_count': 1346.2521832038035,
   'w2_weighted_count': 183.11133163388735,
   'w3_weighted_count': 81.46212516257658},
  'NET': {'w1_weighted_count': 123.62775793856216,
   'w2_weighted_count': 106.99964952263744,
   'w3_weighted_count': 208.08486189059028},
  'VET': {'w1_weighted

In [ ]:
swiss_map = gpd.read_file('./geo-data/landesforstinventar-kantone_2056.geojson')

# Ensure the KantonId in GeoDataFrame is treated as a string if your details keys are strings
swiss_map['KantonId'] = swiss_map['KantonId'].astype(str)

# Merge the details into the GeoDataFrame
# This step adds a new column 'details' to the GeoDataFrame
swiss_map['details'] = swiss_map['KantonId'].map(details)

# Check the resulting GeoDataFrame
swiss_map.to_file('./geo-data/map_data.geojson', driver="GeoJSON")

swiss_map.head(5)

,KantonId,GeometrieId,KantonName_de,KantonName_fr,KantonName_it,KantonName_en,alternateName,geometry,details
0,1,1,Zürich,Zurich,Zurigo,Zurich,ZH,"POLYGON ((2692443.001 1281183.001, 2692994.001...",{'GE': {'w1_weighted_count': 2822.459832175372...
1,2,2,Bern,Berne,Berna,Bern,BE,"MULTIPOLYGON (((2573533.001 1194765.001, 25740...",{'GE': {'w1_weighted_count': 2234.584649055655...
2,3,3,Luzern,Lucerne,Lucerna,Lucerne,LU,"POLYGON ((2662029.001 1237691.001, 2662264.001...",{'GE': {'w1_weighted_count': 734.2253551947114...
3,4,4,Uri,Uri,Uri,Uri,UR,"POLYGON ((2688457.001 1203340.001, 2688740.001...",{'GE': {'w1_weighted_count': 34.61377670488978...
4,5,5,Schwyz,Schwytz,Svitto,Schwyz,SZ,"POLYGON ((2704723.001 1229925.001, 2706210.001...",{'GE': {'w1_weighted_count': 243.1135937638405...


In [ ]:
details[4]

KeyError: 4